# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install .`

In [2]:
%pip install .

Note: you may need to restart the kernel to use updated packages.


ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
You should consider upgrading via the 'c:\Users\erikj\surfdrive\SoDa\projects\synthetic_data\meta-synth\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
# import required packages
import datetime as dt
import pandas as pd
from metasynth import MetaDataset, MetaVar
from metasynth.distribution import DiscreteUniformDistribution

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [4]:
df = pd.read_csv("demonstration.csv")
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,1922-04-08,14:31:52.636968,2022-08-14 02:48:29.857190
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,1906-03-26,11:51:04.236820,2022-08-10 10:25:22.268020
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,1939-09-13,16:56:06.377704,2022-08-13 16:36:00.552160
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,1915-01-03,15:33:58.318343,2022-07-27 19:59:20.559818
4,5,"Allen, Mr. William Henry",male,35.0,0,8.0500,NaN,S,1937-02-09,17:46:53.157645,2022-08-10 02:14:27.126593


MetaSynth will automatically generate the metadata from this DataFrame object so it is important to __ensure the data types for all the variables are correct__. For example, in the dataset above we see that Age is a floating point number whereas it should be an integer (22 instead of 22.0). In addition, there are a few categorical variables (Sex, Parch, Embarked) which are loaded in as string data types.

In general, we support [pandas dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes). For our example dataset we can specify the `dtypes` and load the dataset as follows:

In [5]:
dtypes = {
    "Survived": "category",  # Categories should be assigned this type.
    "Name": "string",  # Strings should be assigned like this
    "Age": "Int64",  # Integer columns that have NA's in them should be explicitly nullable integers.
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category",
}
df = pd.read_csv("demonstration.csv", dtype=dtypes)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22,0,7.2500,<NA>,S,1922-04-08,14:31:52.636968,2022-08-14 02:48:29.857190
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,0,71.2833,C85,C,1906-03-26,11:51:04.236820,2022-08-10 10:25:22.268020
2,3,"Heikkinen, Miss. Laina",female,26,0,7.9250,<NA>,S,1939-09-13,16:56:06.377704,2022-08-13 16:36:00.552160
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,0,53.1000,C123,S,1915-01-03,15:33:58.318343,2022-07-27 19:59:20.559818
4,5,"Allen, Mr. William Henry",male,35,0,8.0500,<NA>,S,1937-02-09,17:46:53.157645,2022-08-10 02:14:27.126593


### Dates, times, and datetimes

One exception to the above is dates, times, and datetimes. Here, we use the types from the built-in `datetime` package. Thus we have to manually transform the strings in the columns with date, time, and datetimes to their proper objects. Since the columns in our example dataset follow the standard ISO-format, we can convert them with the `fromisoformat` method. If they are written in a different format, check out the [datetime library documentation](https://docs.python.org/3/library/datetime.html) on how to convert the strings to datetime/time/date objects.

In [6]:
df["Birthday"] = [dt.date.fromisoformat(x) for x in df["Birthday"]]
df["Board time"] = [dt.time.fromisoformat(x) for x in df["Board time"]]
df["Married since"] = [dt.datetime.fromisoformat(x) for x in df["Married since"]]

Now, let's check the data types of our DataFrame:

In [7]:
df.dtypes

PassengerId               int64
Name                     string
Sex                    category
Age                       Int64
Parch                  category
Fare                    float64
Cabin                    string
Embarked               category
Birthday                 object
Board time               object
Married since    datetime64[ns]
dtype: object

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [8]:
df["Birthday"][0]

datetime.date(1922, 4, 8)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [9]:
meta_dataset = MetaDataset.from_dataframe(df)

c:\users\erikj\surfdrive\soda\projects\synthetic_data\meta-synth\metasynth\disttree.py:119: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [10]:
print(meta_dataset)

# Rows: 891
# Columns: 11

{'name': 'PassengerId', 'description': None, 'type': 'MetaVar', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'MetaVar', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'MetaVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'MetaVar', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'MetaVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'lab

From the `meta_dataset` object, we can now generate synthetic data:

In [11]:
# generate 5 synthetic rows. Rerun the cell to see it change!
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,775,Fp6:83+p_OqXK`wqvH3U{,female,48,0,50.826015,<NA>,C,1926-02-21,16:19:21.197391,2022-07-15 17:16:57.893372126
1,387,aF`@1XN]K-jI#%GHT.c=* IwN5_l%53C']</)r]{'nAWYW...,female,41,0,5.619255,<NA>,S,1938-06-11,11:55:13.072385,2022-07-20 04:45:33.359377118
2,547,"pmaY_;MOF(a9Mu34!^ Bh^=,0",female,<NA>,0,5.810469,<NA>,S,1937-11-01,18:21:10.337937,2022-08-13 21:18:13.751593372
3,680,"(-n J=w)X2C3o^Ai0>/hp5""Lo0 XYP7aT3~ o !,JYON_a",male,61,0,23.142093,<NA>,C,1937-03-29,17:15:29.004066,2022-08-02 03:29:48.454876934
4,860,+xwJ1XDNtr @~Kk#L`P6o/|8: w^m2q~NHc ^K)b~'EM ...,female,30,0,17.410828,<NA>,S,1937-07-21,13:04:46.782128,2022-07-16 06:40:11.903512445


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [12]:
# First, we create a specification dictionary for the variables
var_spec = {"PassengerId": {"unique": True}}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'int64',
 'prop_missing': 0.0,
 'distribution': {'name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [13]:
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"hzoZE>4F._L, ""M2>",female,74,0,71.221457,<NA>,S,1937-05-29,18:34:57.403581,2022-08-02 01:33:59.597053017
1,2,85K%8k2.phY/Qh[ ZL CY3[uC4sMV9-{m^Z*PU.9uKsOEv...,female,35,1,32.096270,Gt7r?M*I8,S,1912-08-06,12:14:24.886557,2022-08-08 02:56:43.738901754
2,3,IWX[{):}(C|8 M_WAVzfl8kDGAkz =~^p,female,33,0,13.506398,G/L3-x,S,1928-12-09,13:01:35.556837,2022-07-28 23:55:28.396955252
3,4,dLI/[L $8&%npmv?1[ +/p!>yq{@/,male,14,1,50.920416,<NA>,S,1925-06-21,12:19:39.416347,2022-08-06 07:22:43.127768522
4,5,;hmo#+x H \A=Po8LaCqy(,female,42,0,28.086279,Fp> 8P7Cf+?j}K,S,1928-01-09,12:51:05.800161,2022-07-24 03:39:03.136358235


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [16]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Christopher Franklin,female,44,0,4.094129,"AP""x""SSh+6!",C,1924-09-19,13:03:05.974780,2022-08-01 06:30:23.003071042
1,2,Taylor Simpson,male,<NA>,0,21.412575,<NA>,S,1938-11-07,17:38:46.003455,2022-07-24 20:28:05.226264635
2,3,Zachary Zuniga,female,<NA>,0,25.549667,<NA>,S,1937-01-30,17:29:46.315289,2022-07-24 19:04:00.907838228
3,4,Virginia King,female,79,0,12.527361,<NA>,Q,1911-07-31,14:07:19.564362,2022-07-21 15:31:45.576022196
4,5,Brittany Lam,male,<NA>,1,9.213059,<NA>,S,1936-07-01,13:27:33.254331,2022-07-18 11:00:08.801569760


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [22]:
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Janet Jefferson,female,<NA>,0,0.800382,T Xqq.t/,C,1903-10-23,11:55:26.858673,2022-08-06 22:25:28.358007754
1,2,James Dunn,male,26,0,0.276690,<NA>,S,1930-05-22,11:05:46.673302,2022-07-28 12:08:31.519309130
2,3,Matthew Padilla,male,33,0,1.509262,<NA>,S,1934-09-10,16:08:13.385991,2022-07-16 16:10:45.805630408
3,4,Susan Carter,male,31,0,1.526071,<NA>,C,1927-04-18,15:08:29.812542,2022-07-17 22:28:42.960296893
4,5,Maria Stephens,female,30,0,0.563327,"B5E9""`Osn*H",S,1915-06-24,15:45:46.271722,2022-08-11 23:38:24.139537071


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can encapsulate this as follows:

In [28]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution([(r"[ABCDEF]", 1), (r"\d{2,3}", 0.9)])  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Terri Myers,male,35,0,2.664029,<NA>,S,1915-11-13,13:57:05.594213,2022-07-16 22:13:46.281040189
1,2,Cheryl Garcia,male,29,0,0.792244,F73,S,1911-07-10,16:09:43.433202,2022-07-31 07:11:18.921941367
2,3,Nicholas Peterson,female,<NA>,0,0.267845,<NA>,C,1936-07-30,11:53:51.297766,2022-08-13 17:59:04.760408932
3,4,William Owens,female,23,1,2.341054,<NA>,Q,1921-12-31,16:06:50.849054,2022-07-29 13:00:06.065806391
4,5,Edward Little,female,<NA>,0,0.463897,<NA>,S,1907-06-04,12:27:24.903537,2022-08-06 02:44:41.118001547
